In [1]:
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

In [2]:

#load the pdf files from the path
loader = DirectoryLoader('data/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

#split text into chunks
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [3]:
#create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={'device':"cpu"})


c:\Users\jetha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#vectorstore
vector_store = FAISS.from_documents(text_chunks,embeddings)


In [5]:
#create llm
llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",
                    config={'max_new_tokens':256,'temperature':0.05})

In [6]:


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)

In [10]:
import pickle

# save vector_store object
with open('vector_store.pkl', 'wb') as f:
    pickle.dump(vector_store, f)


In [ ]:
import pickle
from langchain.chains import ConversationalRetrievalChain

# load vector_store object from pickle file
with open('vector_store.pkl', 'rb') as f:
    vector_store = pickle.load(f)

# create new chain object using the loaded vector_store object
chain = ConversationalRetrievalChain.from_retriever(retriever=vector_store.as_retriever(search_kwargs={"k":2}))
